<a href="https://colab.research.google.com/github/vallika123/Vallika/blob/main/Simple_Robot_game_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import time

#from Node import *
#from Robot import *
import math as Math
import sys

class Robot():


    def __init__(self, _x, _y, _grid):
        self.x = _x
        self.y = _y
        self.maze = []
        self.exits = []

        rows = len(_grid)
        cols = len(_grid[0])

        for r in range(rows):
            row = []
            for c in range(cols):
                row.append(0)
            self.maze.append(row)

        for i in range(rows):
            for j in range(cols):
                if _grid[i][j] == '.':
                    self.maze[i][j] = 0
                elif _grid[i][j] == '#':
                    self.exits.append((i, j))
                    self.maze[i][j] = 1
                else:
                    self.maze[i][j] = 1

        x, y = self.Find_Nearest_Exit()
        self.maze[x][y] = 0



    def Find_Nearest_Exit(self):
        min_dist = sys.maxsize

        for exit_candidate in self.exits:
            exit_x = exit_candidate[0]
            exit_y = exit_candidate[1]

            d = Math.fabs(exit_x - self.x) + Math.fabs(exit_y - self.y)
            if d < min_dist:
                min_dist = d
                self.nearest_exit_x = exit_x
                self.nearest_exit_y = exit_y

        return self.nearest_exit_x, self.nearest_exit_y


    def Move(self):
        path = astar(self.maze, (self.x, self.y), (self.nearest_exit_x, self.nearest_exit_y))
        # print (path)
        if len(path) > 1:
            next = path[1]
            self.x = next[0]
            self.y = next[1]

    def Get_Current_Position(self):
        return (self.x, self.y)

    def Get_Maze(self):
        return self.maze

    def Has_Exited(self):
        exited = (self.x == self.nearest_exit_x) & (self.y == self.nearest_exit_y)
        return exited

class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position


def astar(maze, start, end):
    """Returns a list of tuples as a path from the given start to the given end in the given maze"""

    # Create start and end node
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_node)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1] # Return reversed path

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent squares

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable terrain
            if maze[node_position[0]][node_position[1]] != 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)




class Environment(object):
    """This class contains the simulation environment. """

    NumberOfRobots = 0
    NumberOfExits = 0
    Gridx = 0
    Gridy = 0

    def validate_grid_and_place_robots(self,start_row, end_row, start_col, end_col):
        """Will check the row or column to see if it is safe to place a robots there"""
        global grid
        global robot_positions

        all_valid = True
        for r in range(start_row, end_row):
            for c in range(start_col, end_col):
                if grid[r][c] != ".":
                    all_valid = False
                    break
        if all_valid:
            robot_positions.append([start_row, start_col])
            for r in range(start_row, end_row):
                for c in range(start_col, end_col):
                    grid[r][c] = "O"
        return all_valid


    def validate_grid_and_place_exits(self,start_row, end_row, start_col, end_col):
        """Will check the row or column to see if it is safe to place a robots there"""
        global grid
        global exits_positions

        all_valid = True
        for r in range(start_row, end_row):
            for c in range(start_col, end_col):
                if grid[r][c] != ".":
                    all_valid = False
                    break
        if all_valid:
            exits_positions.append([start_row, start_col])
        for r in range(start_row, end_row):
            for c in range(start_col, end_col):
                grid[r][c] = "#"
        return all_valid


    def try_to_place_robots_on_grid(self,row, col):
        """Based on direction will call helper method to try and place a robots on the grid"""
        start_row, end_row, start_col, end_col = row, row + 1, col, col + 1
        return self.validate_grid_and_place_robots(start_row, end_row, start_col, end_col)


    def try_to_place_exits_on_grid(self,row, col):
        """Based on direction will call helper method to try and place a robots on the grid"""
        start_row, end_row, start_col, end_col = row, row + 1, col, col + 1
        return self.validate_grid_and_place_exits(start_row, end_row, start_col, end_col)

    def print_grid(self):
        global grid
        debug_mode = True
        for row in range(len(grid)):
            for col in range(len(grid[row])):
                if grid[row][col] == "O":
                    if debug_mode:
                        print("O", end=" ")
                    elif grid[row][col] == ".":
                        print(".", end=" ")
                    else:
                        print("#", end=" ")
                else:
                    print(grid[row][col], end=" ")
            print("")
        print(" ", end=" ")

    def create_grid(self, val1,val,robots,exits):
        """Will create a 10x10 grid and randomly place down robots
           of different sizes in different directions"""
        global grid

        num_of_robot = robots
        num_of_exits = exits
        global robot_positions
        global exits_positions
        random.seed(time.time())
        rows, cols = (val, val1)
        grid = []

        for r in range(rows):
            row = []
            for c in range(cols):
                row.append(".")
            grid.append(row)

        num_of_robots_placed = 0
        num_of_exits_placed = 0

        robot_positions = []
        exits_positions = []

        while num_of_robots_placed != num_of_robot:
            random_row = random.randint(0, rows - 1)
            random_col = random.randint(0, cols - 1)
            if self.try_to_place_robots_on_grid(random_row, random_col):
                # our robot has been placed on the grid
                position = (random_row, random_col)
                robot_positions.append(position)
                num_of_robots_placed += 1
        while num_of_exits_placed != num_of_exits:
            for x in range(num_of_exits):
                exit_x = input("Enter x-axis for Exit# "+str(num_of_exits_placed+1)+":")
                exit_y = input("Enter y-axis for Exit# "+str(num_of_exits_placed+1)+":")
                if self.try_to_place_exits_on_grid(int(exit_x), int(exit_y)):
                    num_of_exits_placed += 1
                else:
                    print("Cannot place an exit at ({0},{1})", exit_x, exit_y)


def main():
    """Main entry point of application t54hat runs the game loop"""

    global robot_positions
    global robots
    global gridrows
    global gridcols

    e = Environment()

    gridrows = int(input("Enter number of rows: "))
    gridcols = int(input("Enter number of columns: "))
    robotcount = int(input("Enter number of robots: "))
    exitcount = int(input("Enter number of exits: "))

    #gridrows = 20
    #gridcols = 20
    #robotcount = 3
    #exitcount = 2
    robots = []
    cycle = 0

    # e.create_grid(10, 10, 2, 2)
    e.create_grid(gridrows, gridcols, robotcount, exitcount)

    print("\n\nCycle: ", cycle)
    e.print_grid()

    for i in range(robotcount):
        loc =  robot_positions[2*i + 1]
        loc_x = loc[0]
        loc_y = loc[1]
        robot = Robot(loc_x, loc_y, grid)
        robots.append(robot)
        robot = []

    # while loop that prints grid after every second
    while robotcount > 0:
        time.sleep(1)
        cycle = cycle + 1
        print("\n\nCycle: ", cycle)

        # update the grid
        # get new position
        for robot in robots:
            loc_x_prev, loc_y_prev = robot.Get_Current_Position()

            robot.Move()

            loc_x, loc_y = robot.Get_Current_Position()

            # check if the robot has exitted
            if not robot.Has_Exited():
                grid[loc_x_prev][loc_y_prev] = '.'
                grid[loc_x][loc_y] = 'O'
            else:
                robotcount = robotcount - 1
                grid[loc_x_prev][loc_y_prev] = '.'
                grid[loc_x][loc_y] = '#'
                robots.remove(robot)

        e.print_grid()

if __name__ == "__main__":
    main()


16

Enter number of rows: 10
Enter number of columns: 10
Enter number of robots: 1
Enter number of exits: 2
Enter x-axis for Exit# 1:2
Enter y-axis for Exit# 1:3
Enter x-axis for Exit# 2:5
Enter y-axis for Exit# 2:4


Cycle:  0
. . . . O . . . . . 
. . . . . . . . . . 
. . . # . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . # . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
  

Cycle:  1
. . . . . . . . . . 
. . . . O . . . . . 
. . . # . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . # . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
  

Cycle:  2
. . . . . . . . . . 
. . . O . . . . . . 
. . . # . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . # . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
  

Cycle:  3
. . . . . . . . . . 
. . . . . . . . . . 
. . . # . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 

16